In [13]:
import xmltodict
from collections import Counter

annot_path = "/wynton/group/andrews/data/MrOS/mros-sof_mjhe/vs/EDF/bi/bi0002.edf.XML"
with open(annot_path, encoding='utf-8') as f:
        info_dict = xmltodict.parse(f.read())

events = info_dict['CMPStudyConfig']['ScoredEvents']['ScoredEvent']
sleep_stages = info_dict['CMPStudyConfig']['SleepStages']['SleepStage']
names = [event['Name'] for event in events if 'Name' in event]
Counter(names)

Counter({'SpO2 desaturation': 434,
         'PLM (Left)': 332,
         'PLM (Right)': 266,
         'Limb Movement (Left)': 187,
         'Limb Movement (Right)': 161,
         'Arousal (ASDA)': 109,
         'Hypopnea': 53,
         'SpO2 artifact': 47,
         'Central Apnea': 3})

In [8]:
import os
import pandas as pd 
pd.set_option('display.max_rows', None)

df = pd.read_csv("/Users/alicealbrecht/Desktop/S0001_psg_metadata_2025-09-05.csv")
print(df.shape)
df = df.rename(columns={
    "BDSPPatientID": "sub_id",
    "SessionID": "session"
})
df['sub_id'] = 'S0001' + df['sub_id'].astype(str)
df['session'] = df['session'].astype(int)
df = df[
    df['StudyType'].str.lower().str.contains("diagnostic|dignostic", na=False) &
    (~df['StudyType'].str.lower().str.contains("oxygen", na=False))
]
print(df.shape)
df['StudyType'].value_counts()

(26328, 14)
(9468, 14)


StudyType
PSG Diagnostic                   8297
PSG Diagnostic                    870
Diagnostic                        263
Diagnostic PSG                     22
Diagnostic                          4
PSG DIAGNOSTIC                      2
Diagnostic Psg                      1
Diagnostic Oral Appliance           1
Expanded EEG, Diagnostic            1
Diagnostic Extended EEG             1
Diagnostic with OA                  1
Diagnostic EtCO2                    1
Diagnostic PSG with arm leads       1
PSG diagnostic                      1
Diagnostic, < 6 hrs                 1
Dignostic                           1
Name: count, dtype: int64

In [11]:
master = pd.read_csv("/Users/alicealbrecht/wynton_data/PSG_Pipeline_Outputs/mastersheets/hsp_mgb_mastersheet_before_diagnostic.csv")
print(master.shape)
master['sub_id'] = master['sub_id'].astype(str)
master['session'] = master['session'].astype(int)
master = master.merge(
    df[['sub_id', 'session', 'StudyType']],
    on=['sub_id', 'session'],
    how='left'
)
master = master[
    master['StudyType'].str.lower().str.contains("diagnostic|dignostic", na=False) &
    (~master['StudyType'].str.lower().str.contains("oxygen", na=False))
]
print(master.shape)
master['StudyType'].value_counts()

(26310, 14)
(9464, 15)


StudyType
PSG Diagnostic                   8293
PSG Diagnostic                    870
Diagnostic                        263
Diagnostic PSG                     22
Diagnostic                          4
PSG DIAGNOSTIC                      2
Diagnostic Psg                      1
Diagnostic EtCO2                    1
Dignostic                           1
Expanded EEG, Diagnostic            1
Diagnostic Extended EEG             1
Diagnostic with OA                  1
Diagnostic Oral Appliance           1
PSG diagnostic                      1
Diagnostic PSG with arm leads       1
Diagnostic, < 6 hrs                 1
Name: count, dtype: int64

In [15]:
master = master.drop(columns= ['StudyType'])
master.shape

(9464, 14)

In [16]:
output_path = "/Users/alicealbrecht/wynton_data/PSG_Pipeline_Outputs/mastersheets/hsp_mgb_mastersheet.csv"
master.to_csv(output_path, index=False)

In [ ]:
master = master[~master['annot_path'].isna()]
print(master.shape)
# Look at all possible annotaiotn file found
possible_annot = master['annot_path'].str.extract(r'ses-\d+_(.*)')[0]
possible_annot.unique()

(9326, 14)


array(['task-psg_annotations.csv', 'task_Xltek.csv', nan], dtype=object)

In [2]:
import pandas as pd
master = pd.read_csv("/Users/alicealbrecht/wynton_data/PSG_Pipeline_Outputs/mastersheets/hsp_mgb_mastersheet.csv")
master['annot_options'] = master['annot_path'].str.extract(r'ses-\d+_(.*)')[0]
master['annot_options'].value_counts()

annot_options
task-psg_annotations.csv    9314
task_Xltek.csv                11
Name: count, dtype: int64

In [3]:
master.shape

(9464, 15)